In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%config Completer.use_jedi = False

In [3]:
import yaml

In [4]:
from pysmFISH.pipeline import Pipeline

In [5]:
from pysmFISH.configuration_files import load_experiment_config_file

In [6]:
from pathlib import Path
import time

In [7]:
experiment_fpath = Path('/fish/work_std/LBEXP20201002_EEL_Mouse_1200um')
date_tag = time.strftime("%y%m%d_%H_%M_%S")
pipeline_run_name = date_tag + '_' + experiment_fpath.stem
run_type = 're-run'
parsing_type = 'reparsing_from_storage'
processing_engine = 'htcondor'

In [8]:
running_pipeline = Pipeline(
                        pipeline_run_name= pipeline_run_name,
                        experiment_fpath= experiment_fpath,
                        run_type= run_type,
                        parsing_type= parsing_type,
                        processing_engine= processing_engine
                )

In [9]:
running_pipeline.create_folders_step()

In [15]:
# Add some of the fields that were missing
config_yaml = load_experiment_config_file(experiment_fpath)
config_yaml['Pipeline'] = 'eel-mouse-brain'
config_yaml['Codebook'] = 'gene_hammming_16_11_6_positive_bits.parquet'
config_yaml['StitchingChannel'] = 'Europium'
config_yaml['Probe_FASTA_name'] = 'mBrain.fasta'
config_yaml_fpath = experiment_fpath / (experiment_fpath.stem + '_config.yaml')
with open(config_yaml_fpath, 'w') as new_config:
    yaml.safe_dump(config_yaml, new_config,default_flow_style=False,sort_keys=False)

In [18]:
running_pipeline.QC_check_experiment_yaml_file_step()

In [19]:
running_pipeline.processing_cluster_init_step()

In [20]:
running_pipeline.client

Client Scheduler: tcp://193.10.16.58:33263 Dashboard: http://193.10.16.58:5260/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [21]:
%%time
if running_pipeline.parsing_type != 'no_parsing':
        running_pipeline.nikon_nd2_parsing_graph_step()

CPU times: user 11min 47s, sys: 2min 1s, total: 13min 49s
Wall time: 3h 15min 3s


In [22]:
%%time
running_pipeline.prepare_processing_dataset_step()
running_pipeline.create_analysis_config_file_from_dataset_step()
running_pipeline.determine_tiles_organization()

CPU times: user 10min 56s, sys: 59.6 s, total: 11min 56s
Wall time: 12min 34s


In [23]:
running_pipeline.data.dataset

,barcode,barcode_length,channel,codebook,experiment_name,experiment_type,fov_acquisition_coords_x,fov_acquisition_coords_y,fov_acquisition_coords_z,fov_name,...,species,start_date,stitching_channel,stitching_type,strain,target_name,tissue,total_fovs,zstack,raw_data_location
0,True,16,Cy5,gene_hammming_16_11_6_positive_bits.parquet,LBEXP20201002_EEL_Mouse_1200um,eel-barcoded,1160.7,2673.8,4046.70,raw_data_fov_0,...,Mus musculus,20201002,Europium,both-beads,CD1,EEL647-16,Brain,724,17,/fish/work_std/LBEXP20201002_EEL_Mouse_1200um/...
1,True,16,Cy5,gene_hammming_16_11_6_positive_bits.parquet,LBEXP20201002_EEL_Mouse_1200um,eel-barcoded,1502.2,2673.1,4057.60,raw_data_fov_1,...,Mus musculus,20201002,Europium,both-beads,CD1,EEL647-16,Brain,724,17,/fish/work_std/LBEXP20201002_EEL_Mouse_1200um/...
2,True,16,Cy5,gene_hammming_16_11_6_positive_bits.parquet,LBEXP20201002_EEL_Mouse_1200um,eel-barcoded,3208.4,2327.7,4063.22,raw_data_fov_10,...,Mus musculus,20201002,Europium,both-beads,CD1,EEL647-16,Brain,724,17,/fish/work_std/LBEXP20201002_EEL_Mouse_1200um/...
3,True,16,Cy5,gene_hammming_16_11_6_positive_bits.parquet,LBEXP20201002_EEL_Mouse_1200um,eel-barcoded,-4987.3,1322.2,4029.94,raw_data_fov_100,...,Mus musculus,20201002,Europium,both-beads,CD1,EEL647-16,Brain,724,17,/fish/work_std/LBEXP20201002_EEL_Mouse_1200um/...
4,True,16,Cy5,gene_hammming_16_11_6_positive_bits.parquet,LBEXP20201002_EEL_Mouse_1200um,eel-barcoded,-4645.8,1321.4,4030.64,raw_data_fov_101,...,Mus musculus,20201002,Europium,both-beads,CD1,EEL647-16,Brain,724,17,/fish/work_std/LBEXP20201002_EEL_Mouse_1200um/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23163,True,16,Europium,gene_hammming_16_11_6_positive_bits.parquet,LBEXP20201002_EEL_Mouse_1200um,eel-barcoded,-6694.2,1326.0,4027.34,raw_data_fov_95,...,Mus musculus,20201002,Europium,both-beads,CD1,Europium,Brain,724,17,/fish/work_std/LBEXP20201002_EEL_Mouse_1200um/...
23164,True,16,Europium,gene_hammming_16_11_6_positive_bits.parquet,LBEXP20201002_EEL_Mouse_1200um,eel-barcoded,-6353.0,1325.3,4028.16,raw_data_fov_96,...,Mus musculus,20201002,Europium,both-beads,CD1,Europium,Brain,724,17,/fish/work_std/LBEXP20201002_EEL_Mouse_1200um/...
23165,True,16,Europium,gene_hammming_16_11_6_positive_bits.parquet,LBEXP20201002_EEL_Mouse_1200um,eel-barcoded,-6011.3,1324.6,4028.92,raw_data_fov_97,...,Mus musculus,20201002,Europium,both-beads,CD1,Europium,Brain,724,17,/fish/work_std/LBEXP20201002_EEL_Mouse_1200um/...
23166,True,16,Europium,gene_hammming_16_11_6_positive_bits.parquet,LBEXP20201002_EEL_Mouse_1200um,eel-barcoded,-5670.1,1323.9,4029.48,raw_data_fov_98,...,Mus musculus,20201002,Europium,both-beads,CD1,Europium,Brain,724,17,/fish/work_std/LBEXP20201002_EEL_Mouse_1200um/...


In [24]:
running_pipeline.create_running_functions_step()

In [25]:
running_pipeline.running_functions

{'fish_channels_preprocessing': 'standard_not_norm_preprocessing',
 'fish_channels_dots_calling': 'osmFISH_peak_based_detection_fast',
 'reference_channels_dots_calling': 'osmFISH_peak_based_detection_fast',
 'fresh_sample_reference_preprocessing': 'large_beads_preprocessing',
 'fresh_sample_reference_dots_calling': 'osmFISH_peak_based_detection_fast',
 'fresh_sample_nuclei_preprocessing': 'fresh_nuclei_filtering',
 'reference_channels_preprocessing': 'both_beads_preprocessing'}

In [26]:
# Test registration using only large beads
running_pipeline.running_functions['reference_channels_preprocessing'] = 'standard_not_norm_preprocessing'

In [ ]:
# Check the pipeline analysis config file

In [ ]:
%%time
running_pipeline.processing_barcoded_eel_step()

In [30]:
%%time
running_pipeline.QC_registration_error_step()

CPU times: user 1h 9min 47s, sys: 4min 6s, total: 1h 13min 53s
Wall time: 1h 8min 45s


In [31]:
%%time
running_pipeline.remove_duplicated_dots_graph_step()

CPU times: user 13min 26s, sys: 17.7 s, total: 13min 43s
Wall time: 13min 48s


In [32]:
running_pipeline.client.close()
running_pipeline.cluster.close()

distributed.deploy.adaptive_core - ERROR - Adaptive stopping due to error in <closed TCP>: Stream is closed: while trying to call remote method 'adaptive_target'
